# Evaluate models on Multi-institute MCL cohorts

In [1]:
import os
import pandas as pd
import numpy as np
import math
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
import scipy.stats
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
pd.set_option('display.max_colwidth', None)

### Mayo, CBM, Brock, radiomics risk
* using precomputed values
* unclear why subset of cohrot has NaN risk for CBM and radiomic models 

In [2]:
cohort_path = "/home/local/VANDERBILT/litz/github/MASILab/DeepLungScreening/cohorts/multi_mcl/multi_mcl_prep_v1.csv"
cohort_df = pd.read_csv(cohort_path)
cohort_df

,pid,id,filename,session,scan_date,cohort,with_image,with_marker,age,race,...,phist,fhist,smo_status,pkyr,cyfra,lung_cancer,mayo_risk,CBM_risk,radiomics_risk,brock_risk
0,376873159,NaN,NaN,NaN,NaN,2.0,False,True,64.0,1.0,...,0,0,0.0,70.0,0.268318,0,0.510189,0.366301,0.742949,0.278794
1,398957338,398957338time20110101,398957338time20110101,0.0,2011-01-01,2.0,True,True,66.0,1.0,...,0,0,1.0,50.0,1.185300,0,0.219309,0.093341,0.335967,0.084037
2,1759365513,1759365513time20100101,1759365513time20100101,0.0,2010-01-01,2.0,True,True,80.0,1.0,...,0,0,0.0,10.0,1.545812,1,0.733087,0.863324,0.795813,0.429047
3,2368954232,2368954232time20120101,2368954232time20120101,0.0,2012-01-01,2.0,True,True,52.0,1.0,...,0,0,1.0,50.0,3.212954,1,0.749581,0.966498,0.822853,0.588785
4,2962357115,2962357115time20110101,2962357115time20110101,0.0,2011-01-01,2.0,True,True,68.0,1.0,...,0,0,0.0,94.5,0.995075,0,0.070825,0.039126,0.299580,0.009568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,20449484282,20449484282time20140218,20449484282time20140218,1.0,2014-02-18,3.0,True,True,77.0,NaN,...,0,0,0.0,94.5,0.046962,1,0.954705,NaN,NaN,0.614620
1457,20719819907,NaN,NaN,NaN,NaN,3.0,False,True,76.0,NaN,...,0,0,1.0,29.5,1.489888,0,0.503275,NaN,NaN,0.272698
1458,19182262002,19182262002time20140225,19182262002time20140225,0.0,2014-02-25,3.0,True,True,51.0,NaN,...,0,0,1.0,30.0,0.083823,1,0.363386,NaN,NaN,0.163168
1459,19182262002,19182262002time20140509,19182262002time20140509,1.0,2014-05-09,3.0,True,True,51.0,NaN,...,0,0,1.0,30.0,0.083823,1,0.363386,NaN,NaN,0.163168


### merge CSCNN results

In [3]:
cs_path = "/home/local/VANDERBILT/litz/github/MASILab/DeepLungScreening/cohorts/multi_mcl/multi_mcl_latest_scan_v1.csv"
cscnn_path = "/home/local/VANDERBILT/litz/github/MASILab/DSB2017/models/config_multi_mcl_1115/preds.csv"
cscnn_pred = pd.read_csv(cscnn_path)
cs = pd.read_csv(cs_path, dtype={'pid':str})
cs['cscnn_risk'] = cscnn_pred['pred']
cohort_df = cohort_df.merge(cs[['pid', 'cscnn_risk']], on='pid', how='left')


### merge tdvit results

In [4]:
twoscan_path = "/home/local/VANDERBILT/litz/github/MASILab/DeepLungScreening/cohorts/multi_mcl/multi_mcl_2scan.csv"
tdvit_path = "/home/local/VANDERBILT/litz/github/MASILab/time-distance-transformer/models/1109_multi_mcl_td/pred.csv"
twoscan = pd.read_csv(twoscan_path, dtype={'pid':str})
tdvit_pred = pd.read_csv(tdvit_path, dtype={'PID':str})

twoscan = twoscan.merge(tdvit_pred[['PID', 'pred']], left_on='pid', right_on='PID')
twoscan = twoscan.rename(columns={'pred':'tdvit_risk'})
cohort_df = cohort_df.merge(twoscan[['pid', 'id', 'tdvit_risk']], on=['pid', 'id'], how='left')

In [5]:
# remove nulls
nonnull_cohort = cohort_df[~cohort_df['CBM_risk'].isnull() & ~cohort_df['mayo_risk'].isnull() 
    & ~cohort_df['brock_risk'].isnull() & ~cohort_df['radiomics_risk'].isnull() 
    & ~cohort_df['cscnn_risk'].isnull()]
print(nonnull_cohort['cohort'].value_counts())


1.0    402
3.0    159
2.0     96
4.0     81
Name: cohort, dtype: int64


### merge DLSTM

In [6]:
twoscan_path = "/home/local/VANDERBILT/litz/github/MASILab/DeepLungScreening/cohorts/multi_mcl/multi_mcl_2scan.csv"
dlstm_path = "/home/local/VANDERBILT/litz/github/MASILab/RNNLung/compare/tumor_NLST/models/1130_multi_mcl_vumc/pred.csv"
twoscan = pd.read_csv(twoscan_path, dtype={'pid':str})
dlstm_pred = pd.read_csv(dlstm_path, dtype={'pid':str})


twoscan = twoscan.merge(dlstm_pred[['pid', 'pred']], on='pid')
twoscan = twoscan.rename(columns={'pred':'dlstm_risk'})
cohort_df = cohort_df.merge(twoscan[['pid', 'id', 'dlstm_risk']], on=['pid', 'id'], how='left')


### Eval cross sectional models

In [7]:
random_seed=56
n_bootstrap = 1000 # number of bootstrap samples with replacement (the samples are same size as cohort)

cohort_names = ['VUMC', 'UPMC', 'DECAMP', 'UCD']
model_names = ['CBM', 'mayo', 'brock', 'radiomics', 'cscnn']

def compute_model_auc(sample, model):
    y = sample['lung_cancer']
    y_prob = sample[f"{model}_risk"]
    return roc_auc_score(y, y_prob)

def compute_ci(data, confidence=0.95):
    a = 1.0*np.array(data)
    n = len(a)
    mu, se = np.mean(a), scipy.stats.sem(a)
    h = se*scipy.stats.t.ppf((1+confidence)/2.0, n-1)
    # print(mu)
    return mu, mu-h, mu+h

dfrows= []

for i, cohort_name in enumerate(cohort_names):
    cohort = nonnull_cohort[nonnull_cohort['cohort']==i+1]
    for model in model_names:
        # calculate 95% CI with bootstrap sampling
        aucs = []
        for i in range(n_bootstrap):
            sample = cohort.sample(frac=1.0, replace=True)
            aucs.append(compute_model_auc(sample, model))

        mean_auc, ci_low, ci_high = compute_ci(aucs)
        dfrows.append({'cohort': cohort_name, 'model': model, 'mean_AUC': mean_auc, 'ci_low':ci_low, 'ci_high':ci_high, 'std': np.std(aucs)})

metrics = pd.DataFrame(dfrows)
metrics

,cohort,model,mean_AUC,ci_low,ci_high,std
0,VUMC,CBM,0.861933,0.860757,0.863109,0.018940
1,VUMC,mayo,0.749726,0.747789,0.751663,0.031198
2,VUMC,brock,0.745992,0.744023,0.747961,0.031711
3,VUMC,radiomics,0.778632,0.776818,0.780446,0.029215
4,VUMC,cscnn,0.663570,0.661631,0.665509,0.031234
5,UPMC,CBM,0.943905,0.942597,0.945214,0.021083
6,UPMC,mayo,0.857421,0.855020,0.859821,0.038666
7,UPMC,brock,0.872919,0.870809,0.875030,0.033996
8,UPMC,radiomics,0.874189,0.871853,0.876524,0.037617
9,UPMC,cscnn,0.781145,0.778153,0.784138,0.048200


### Eval Longitudinal models

In [8]:
# remove nulls
nonnull_cohort = cohort_df[~cohort_df['tdvit_risk'].isnull()]
print(nonnull_cohort['cohort'].value_counts())

1.0    286
3.0     92
Name: cohort, dtype: int64


In [9]:
nonnull_cohort

,pid,id,filename,session,scan_date,cohort,with_image,with_marker,age,race,...,pkyr,cyfra,lung_cancer,mayo_risk,CBM_risk,radiomics_risk,brock_risk,cscnn_risk,tdvit_risk,dlstm_risk
232,2150250843,2150250843time20120801,2150250843time20120801,0.0,2012-08-01,1.0,True,False,62.0,1.0,...,80.0,NaN,0,0.200897,NaN,NaN,0.011388,0.148497,0.253520,0.327980
233,2150250843,2150250843time20170728,2150250843time20170728,1.0,2017-07-28,1.0,True,False,62.0,1.0,...,80.0,NaN,0,0.200897,NaN,NaN,0.011388,0.148497,0.253520,0.327980
235,5828995798,5828995798time20130604,5828995798time20130604,0.0,2013-06-04,1.0,True,False,57.0,1.0,...,64.5,NaN,0,0.094610,NaN,NaN,0.011377,0.365323,0.348641,0.197371
236,5828995798,5828995798time20161031,5828995798time20161031,1.0,2016-10-31,1.0,True,False,57.0,1.0,...,64.5,NaN,0,0.094610,NaN,NaN,0.011377,0.365323,0.348641,0.197371
238,10820157066,10820157066time20131113,10820157066time20131113,1.0,2013-11-13,1.0,True,False,64.0,1.0,...,75.0,NaN,0,0.064962,NaN,NaN,0.010984,0.215101,0.227476,0.228910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,28656371809,28656371809time20160217,28656371809time20160217,2.0,2016-02-17,3.0,True,True,55.0,NaN,...,51.0,0.260293,1,0.410718,NaN,NaN,0.108775,0.924885,0.729894,0.663913
1455,20449484282,20449484282time20130506,20449484282time20130506,0.0,2013-05-06,3.0,True,True,77.0,NaN,...,94.5,0.046962,1,0.954705,NaN,NaN,0.614620,0.281178,0.322683,0.065404
1456,20449484282,20449484282time20140218,20449484282time20140218,1.0,2014-02-18,3.0,True,True,77.0,NaN,...,94.5,0.046962,1,0.954705,NaN,NaN,0.614620,0.281178,0.322683,0.065404
1459,19182262002,19182262002time20140509,19182262002time20140509,1.0,2014-05-09,3.0,True,True,51.0,NaN,...,30.0,0.083823,1,0.363386,NaN,NaN,0.163168,0.785776,0.764752,0.531484


In [10]:
random_seed=56
n_bootstrap = 1000 # number of bootstrap samples with replacement (the samples are same size as cohort)

cohort_names = [(1, 'VUMC'), (3, 'DECAMP')]
model_names = ['tdvit', 'dlstm']

dfrows= []

for i, cohort_name in cohort_names:
    cohort = nonnull_cohort[nonnull_cohort['cohort']==i]
    for model in model_names:
        # calculate 95% CI with bootstrap sampling
        aucs = []
        for i in range(n_bootstrap):
            sample = cohort.sample(frac=1.0, replace=True)
            aucs.append(compute_model_auc(sample, model))

        mean_auc, ci_low, ci_high = compute_ci(aucs)
        dfrows.append({'cohort': cohort_name, 'model': model, 'mean_AUC': mean_auc, 'ci_low':ci_low, 'ci_high':ci_high, 'std', np.std(aucs)})

metrics = pd.DataFrame(dfrows)
metrics

SyntaxError: invalid syntax (319455707.py, line 19)